## 论文解析/分析
1. 读pdf
2. LLM总结信息&结构化输出
3. LLM分析适配性

In [ ]:
# Install required packages
! pip install pydantic langchain langchain-community PyPDF2 python-dotenv tqdm

In [ ]:
# Import necessary modules
from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.llms import Tongyi
import os
import json
from PyPDF2 import PdfReader
from datetime import datetime
from tqdm import tqdm
from dotenv import load_dotenv

In [ ]:
# get Dashscope key
# 加载 .env 文件
load_dotenv()
dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")
# print(dashscope_api_key)

读写文件

In [ ]:
# 读取指定文件夹中的所有 PDF 文件内容
def read_papers_from_folder(folder_path):
    papers = []
    
    # 获取所有 PDF 文件路径
    pdf_files = [filename for filename in os.listdir(folder_path) if filename.endswith(".pdf")]
    
    # 使用 tqdm 包裹 pdf_files 列表，显示进度条
    for filename in tqdm(pdf_files, desc="Reading PDFs", unit="file"):
        file_path = os.path.join(folder_path, filename)
        try:
            reader = PdfReader(file_path)
            paper_content = ""
            
            # 遍历每一页并提取文本
            for page in reader.pages:
                paper_content += page.extract_text()
            
            papers.append(paper_content)
        
        except Exception as e:
            # 捕获异常并打印错误信息
            tqdm.write(f"Error reading {filename}: {e}")  # 使用 tqdm.write 避免干扰进度条
    
    return papers

# 将分析结果保存到 JSON 文件中
def save_results_to_json(results, output_file):
    # 创建父目录（如果不存在）
    # exist_ok=True 表示如果目录已经存在，函数不会抛出异常，而是直接返回
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # 写入 JSON 文件
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

用于分析的LLM类

In [ ]:
class PaperInfo(BaseModel):
    """
    定义论文信息的结构化输出格式。
    """
    title: str = Field(description="Title of the paper")
    model_type: str = Field(description="Model type of the method, black box or white box")
    model_names: List[str] = Field(description="List of names of the models used in the method")
    method_description: str = Field(description="Description of the attack method")
    main_result: str = Field(description="Main result of the paper")

class AnalyzeLLM:
    def __init__(self, model_name, api_key):
        # 初始化大语言模型
        self.llm = Tongyi(model=model_name, api_key=api_key)
        
        # 使用 PydanticOutputParser 来解析结构化输出
        self.output_parser = PydanticOutputParser(pydantic_object=PaperInfo)
        
        # 定义提示模板，用于引导模型生成结构化输出
        self.parse_prompt_template = PromptTemplate(template="""
        Analyze the following paper content and extract structured information:
        {content}
        
        Please provide the output in the following format:
        {format_instructions}
        """, input_variables=["content"], partial_variables={"format_instructions": self.output_parser.get_format_instructions()})
    
    def analyze(self, content: str):
        """
        分析论文内容并提取结构化信息。
        """
        try:
            # 构造提示
            prompt = self.parse_prompt_template.format(content=content)
            
            # 调用大语言模型获取响应
            response = self.llm.invoke(prompt)
            if not response or not isinstance(response, str):
                raise ValueError("从模型接收到的响应无效。")
            
            # 解析模型输出为结构化数据
            structured_output = self.output_parser.parse(response)
            
            return structured_output
        
        except Exception as e:
            # 捕获异常并返回错误信息
            raise RuntimeError(f"分析论文时发生错误：{str(e)}")

In [ ]:
# 中文版
class PaperInfo_cn(BaseModel):
    """
    定义论文信息的结构化输出格式。
    """
    title: str = Field(description="论文标题")
    model_type: str = Field(description="方法的模型类型，【黑盒】或【白盒】")
    model_names: List[str] = Field(description="方法中使用的模型名称列表")
    method_description: str = Field(description="攻击方法的描述")
    main_result: str = Field(description="论文的主要结果")

class AnalyzeLLM_cn:
    def __init__(self, model_name, api_key):
        # 初始化大语言模型
        self.llm = Tongyi(model=model_name, api_key=api_key, streaming=True)
        
        # 使用 PydanticOutputParser 来解析结构化输出
        self.output_parser = PydanticOutputParser(pydantic_object=PaperInfo_cn)
        
        # 定义提示模板，用于引导模型生成结构化输出
        self.parse_prompt_template = PromptTemplate(template="""
        分析以下论文内容并提取结构化信息：
        {content}
        
        请按照以下格式提供输出：
        {format_instructions}
        """, input_variables=["content"], partial_variables={"format_instructions": self.output_parser.get_format_instructions()})
    
    def analyze(self, content: str):
        """
        分析论文内容并提取结构化信息。
        """
        try:
            # 构造提示
            prompt = self.parse_prompt_template.format(content=content)
            
            # 调用大语言模型获取响应
            response = self.llm.invoke(prompt)
            if not response or not isinstance(response, str):
                raise ValueError("从模型接收到的响应无效。")
            
            # 解析模型输出为结构化数据
            structured_output = self.output_parser.parse(response)
            
            return structured_output
        
        except Exception as e:
            # 捕获异常并返回错误信息
            raise RuntimeError(f"分析论文时发生错误：{str(e)}")

用于评价的LLM  
实测表现不好，不如人工评价是否有用

In [ ]:
class Evaluation(BaseModel):
    """
    定义对论文的评估结果输出格式
    """
    evaluation: str = Field(description="论文中的方法是否适合用于本次比赛，【是】或【否】")
    evaluation_reason: str = Field(description="对论文的评估结果的解释")

class EvaluationLLM:
    def __init__(self, model_name, api_key):
        self.llm = Tongyi(model_name=model_name, api_key=api_key, streaming=True)
        self.parser = PydanticOutputParser(pydantic_object=Evaluation)
        self.prompt = PromptTemplate(
            template="""
                <instruction>
                请根据论文的内容，评估该论文是否适合用于本次比赛。
                </instruction>
                <question>
                比赛题目：
                在第一阶段，我们将提供 6 个风险类别（基本文本）下的有害文本查询，每个类别 30 个查询，共计 180 个查询。对于每个基本文本查询，参与者需要设计一个对抗性文本（不超过 100 个字）和一个对抗性图像，形成图像-文本对。图像-文本对的目标是触发指定的基础模型，为有害文本查询生成安全风险输出。
                </question>
                <paper>
                {paper}
                </paper>
                <evaluation>
                {format_instructions}
                </evaluation>
                """,
            input_variables=["paper"],
            partial_variables={"format_instructions": self.parser.get_format_instructions()}
        )

    def evaluate(self, paper):
        """
        根据论文内容生成评估结果。
        """
        try:
            # 构造提示
            prompt = self.prompt.format(paper=paper)
            
            # 调用大语言模型获取响应
            raw_response = self.llm.invoke(prompt)
            
            if not raw_response or not isinstance(raw_response, str):
                raise ValueError("从模型接收到的响应无效。")
            
            # 解析模型输出为结构化数据
            structured_output = self.parser.parse(raw_response)
            
            return structured_output
        
        except Exception as e:
            # 捕获异常并返回错误信息
            raise RuntimeError(f"评估论文时发生错误：{str(e)}")
    

进行分析

In [ ]:
# 读取论文、分析并保存结果
def process_papers(folder_path, output_file, analyzer, evaluator):
    # 读取论文内容
    print("Reading papers from folder...")
    papers = read_papers_from_folder(folder_path)
    
    # 分析每篇论文并收集结果
    results = []
    print("Analyzing papers...")
    for paper in tqdm(papers, desc="Analyzing Papers", unit="paper"):
        try:
            # 调用 analyzer 对论文进行分析
            structured_info = analyzer.analyze(paper)
            # 调用 evaluator 对论文进行评估（实测表现不行）
            # evaluation_result = evaluator.evaluate(paper)
            # print(structured_info)
            results.append({
                "title": structured_info.title,
                "model_type": structured_info.model_type,
                "model_names": structured_info.model_names,
                "method_description": structured_info.method_description,
                "main_result": structured_info.main_result,
                "evaluation": evaluation_result.evaluation,  # 是否适合用于比赛
                "evaluation_reason": evaluation_result.evaluation_reason  # 评估理由
            })
        except Exception as e:
            tqdm.write(f"Error:{e}")
    
    # 保存结果到 JSON 文件
    print("Saving results to JSON file...")
    save_results_to_json(results, output_file)
    print(f"Results saved to {output_file}")

In [ ]:
# 初始化分析器
analyzer = AnalyzeLLM(model_name="qwen-turbo", api_key=dashscope_api_key)

In [ ]:
# 初始化分析器（中文版）
analyzer_cn=AnalyzeLLM_cn(model_name="qwen-turbo", api_key=dashscope_api_key)

In [ ]:
# 初始化评估器
evaluator=EvaluationLLM("qwen-turbo",dashscope_api_key)

In [ ]:
papers_folder="../data/MLLM_attacks"
output_path="../data/output/paper_infos.json"

process_papers(papers_folder, output_path,analyzer_cn,evaluator)